In [1]:
import numpy as np
import random as rd
#np.seterr(divide='ignore', invalid='ignore')

In [2]:
def define_HMMs(K,R,M):
    # Class probabilities - one class is much more probable than the rest
    pi = np.zeros((K))
    class1 = rd.randint(0,K-1)
    pi[class1] = 0.5
    for k in range(K):
        if pi[k]==0.0:
            pi[k] = 0.5/(K-1)

    # Start probabilities - UNIFORM
    init = 1/R*np.ones((R))

    # Transition probabilities - from each row, there are only two possible next states, with varying probabilities
    A = np.zeros((K, R, R))
    for k in range(K):
        for r in range(R):
            rand = rd.randint(10,20)
            row1 = rd.randint(0,R-1)
            row2 = rd.randint(0,R-1)
            while(row2 == row1):
                row2 = rd.randint(0,R-1)

            A[k,r,row1] = rand/20
            A[k,r,row2] = (20-rand)/20

    # Emission probabilities - different noise for different classes, but same noise for all rows within that class
    E = np.zeros((K, R, R))
    for k in range(K):
        rand = rd.randint(10,20)
        for r in range(R):
            E[k,r,r] = rand/20
            E[k,r,(r+1)%nr_rows] = (20-rand)/40
            E[k,r,(r-1)%nr_rows] = (20-rand)/40

    return pi, init, A, E

def generate_states(k,R,M):
	init = start_prob
	X = np.zeros((M), dtype=int)

	rand = rd.random()
	sum_steps = 0.0
	for r in range(R):
		if rand>=sum_steps and rand<sum_steps+init[r]:
			X[0] = r
			break
		sum_steps += init[r]

	for m in range(1,M):
		A = transition_prob[k,X[m-1],:]
		rand = rd.random()
		sum_steps = 0.0
		for r in range(R):
			if rand>=sum_steps and rand<sum_steps+A[r]:
				X[m] = r;
				break
			sum_steps += A[r]
	
	return X

		
def generate_observations(k,R,M,X):
	Z =  np.zeros((M), dtype=int)
	for m in range(M):
		E = emission_prob[k,X[m],:]
		rand = rd.random()
		sum_steps = 0.0
		for r in range(R):
			if rand>=sum_steps and rand<sum_steps+E[r]:
				Z[m] = r
				break
			sum_steps += E[r]
	
	return Z


def generate_data(N,K,R,M):
	classes = np.zeros((N), dtype=int)
	observations = np.zeros((N,M), dtype=int)

	for n in range(N):
		rand = rd.random()
		sum_steps = 0.0
		for k in range(K):
			if rand>=sum_steps and rand<sum_steps+class_prob[k]:
				k_n = k;
				break
			sum_steps += class_prob[k]

		classes[n] = k_n
		observations[n,:] = generate_observations(k_n, R, M, generate_states(k_n, R, M))

	return classes, observations

def create_AB(K, R, M):
    A = np.zeros((K,R,R))
    B = np.zeros((K,R,R))
    for k in range(K):
        for i in range(R):
            for j in range(R):
                A[k][i][j] = (np.random.randint(R)-0.5)*0.1/R
                B[k][i][j]= 1.0/R + (np.random.randint(R)-0.5)*0.1/R
    A = np.abs(A)
    B = np.abs(B)
    a = A/A.sum(axis=2)[:,:,None]
    b = B/B.sum(axis=2)[:,:,None]
    return a,b


In [3]:
def get_transition(k, r1, r2, A):
    return A[k][r1][r2]

def get_emission(k, r, o, B):
    return B[k][r][o] 

def forward(k, observations, A, B, start_prob):
    #pi, R = get_init()
    pi = start_prob
    alpha = np.zeros((M, R))

    # base case
    O = []
    for r in range(R):
        O.append(get_emission(k, r, observations[0], B))
    alpha[0, :] = pi * O[:]

    # recursive case
    for m in range(1, M):
        for r2 in range(R):
            for r1 in range(R):
                transition = get_transition(k, r1, r2, A)
                emission = get_emission(k, r2, observations[m], B)
                alpha[m, r2] += alpha[m - 1, r1] * transition * emission

    return alpha, np.sum(alpha[M - 1, :])

def backward(k, observations, A, B, start_prob):
    #pi, R = get_init()
    pi = start_prob
    beta = np.zeros((M, R))

    # base case
    beta[M - 1, :] = 1

    # recursive case
    for m in range(M - 2, -1, -1):
        for r1 in range(R):
            for r2 in range(R):
                transition = get_transition(k, r1, r2, A)
                emission = get_emission(k, r2, observations[m + 1], B)
                beta[m, r1] += beta[m + 1, r2] * transition * emission

    O = []
    for r in range(R):
        O.append(get_emission(k, r, observations[0], B))

    return beta, np.sum(pi * O[:] * beta[0, :])

In [9]:
def gamma_cal(k, observations, A, B, start_prob):
    alpha, ha = forward(k, observations, A, B, start_prob)
    beta, _ = backward(k, observations, A, B, start_prob)
    gamma = alpha*beta/ha
    digamma = np.zeros([nr_columns, nr_rows, nr_rows])
    for t in range(nr_columns-1):
        for i in range(nr_rows):
            for j in range(nr_rows):
                digamma[t,i,j] = alpha[t,i]*A[k][i][j]*beta[t+1,j]*B[k][j][observations[t+1]]
        digamma[t] = digamma[t]/np.sum(digamma[t])
    return gamma, digamma
def param_cal(A, B, start_prob):
    gamma_all = np.zeros((nr_classes, nr_vehicles, M, R))
    digamma_all = np.zeros((nr_classes, nr_vehicles, M, R, R))
    for c in range(nr_classes):
        for n in range(nr_vehicles):
            gamma_all[c][n], digamma_all[c][n] = gamma_cal(c, data[n], A, B, start_prob)
    return gamma_all, digamma_all
def cal_MN(r_nc, gamma_all, digamma_all):
    N_clk = np.zeros((nr_classes, R, R))
    M_csk = np.zeros((nr_classes, R, R))
    for c in range(nr_classes):
        for i in range(R):
            for j in range(R):
                x, g = 0, 0
                for n in range(nr_vehicles):
                    y, h = 0, 0
                    for t in range(M):
                        y += digamma_all[c][n][t][i][j]
                        if data[n][t] == j:
                            h += gamma_all[c][n][t][i]
                    x += r_nc[n][c]*y
                    g += r_nc[n][c]*h
                N_clk[c][i][j] = x
                M_csk[c][i][j] = g
    return N_clk, M_csk

#Calculate Responsibilities of clusters
def cal_resp(p_c, A, B, start_prob):
    r_nc = np.zeros((nr_vehicles,nr_classes))
    l_nc = np.zeros((nr_vehicles,nr_classes))
    for i in range(nr_vehicles):
        for c in range(nr_classes):
            _, l_nc[i][c] = forward(c, data[i], A, B, start_prob)
    a = np.dot(p_c, l_nc.T)
    for i in range(nr_vehicles):
        for c in range(nr_classes):
            if a[i] == 0: print("Zero divide coming")
            r_nc[i][c] = p_c[c]*l_nc[i][c]/a[i]
    return r_nc

def value_correct( X):
    a = np.log(X)
    for c in range(nr_classes):
        for s in range(R):
            for k in range(R):
                if a[c,s,k] == np.inf or a[c,s,k] == -np.inf:
                    a[c,s,k] = 0
    return a

def re_estimate(N_clk, M_csk, r_nc):
    nx = np.zeros((nr_classes,R,1))
    mx = np.zeros((nr_classes,R,1))
    A_new = np.zeros((nr_classes,R,R))
    B_new = np.zeros((nr_classes,R,R))
    r_c = np.zeros((nr_classes))
    for c in range(nr_classes):
        for i in range(R):
            nx[c][i] = np.sum(N_clk[c][i])
            mx[c][i] = np.sum(M_csk[c][i])            
    for c in range(nr_classes):
        for i in range(R):
            for j in range(R):
                A_new[c][i][j] = N_clk[c][i][j]/nx[c][i]
                B_new[c][i][j] = M_csk[c][i][j]/mx[c][i]                
    for c in range(nr_classes):
        a = 0
        for n in range(nr_vehicles):
            a += r_nc[n][c]
        r_c[c] = a
    p_c_new = r_c/nr_vehicles
    return p_c_new, A_new, B_new

def Q_eval(p_c, A, B, start_prob):
    r_nc = cal_resp(p_c, A, B, start_prob)
    #print('cal_resp done')
    gamma_all, digamma_all = param_cal(A, B, start_prob)
    gamma_all = np.nan_to_num(gamma_all)
    digamma_all = np.nan_to_num(digamma_all)
    #print('param_cal done')
    N_clk, M_csk = cal_MN(r_nc, gamma_all, digamma_all)
    #print('cal_MN done')
    a = value_correct(A)
    b = value_correct(B)
    logprob = (np.sum(np.dot(r_nc,np.log(p_c))) + np.sum(a*N_clk) + np.sum(b*M_csk))
    return logprob, N_clk, M_csk, r_nc
def train(p_c, A, B, start_prob, oldVal):
    val, N_clk, M_csk, r_nc = Q_eval(p_c, A, B, start_prob)
    #print(val)
    if val > oldVal+0.001:
        oldVal = val
        p_c_new, A_new, B_new = re_estimate(N_clk, M_csk, r_nc)
        print(val)
        return train(p_c_new, A_new, B_new, start_prob, oldVal)
    else:
        print("Converged", val)
        return p_c, A, B    
def get_targets(r_nc):
    new_tar = []
    for i in range(nr_vehicles):
        new_tar.append(np.argmax(r_nc[i]))
    new_tar = np.array(new_tar)
    return new_tar
def form_clusters(arr):
    C = {}
    for i in np.unique(arr):
        C[i] = list(np.where(arr == i))
    return C

In [13]:
#To initialize A and B
def create_AB(K, R, M):
    A = np.zeros((K,R,R))
    B = np.zeros((K,R,R))
    for k in range(K):
        for i in range(R):
            for j in range(R):
                A[k][i][j] = (np.random.randint(R)-0.5)*0.1/R
                B[k][i][j]= 1.0/R + (np.random.randint(R)-0.5)*0.1/R
    A = np.abs(A)
    B = np.abs(B)
    a = A/A.sum(axis=2)[:,:,None]
    b = B/B.sum(axis=2)[:,:,None]
    return a,b

nr_vehicles = 20;
nr_classes = 2;
nr_rows = 10;
nr_columns = 20;
M, R = nr_columns, nr_rows
#class probability
#p_c = np.ones(nr_classes)*(1/nr_classes)

class_prob, start_prob, transition_prob, emission_prob = define_HMMs(nr_classes, nr_rows, nr_columns)
targets, data = generate_data(nr_vehicles, nr_classes, nr_rows, nr_columns)
A, B = create_AB(nr_classes, R, M)
#A = transition_prob
#B = emission_prob
p_c = class_prob
print(A)
print(B)


Observed sequences
 [[6 6 7 0 8 8 8 2 6 6 7 0 8 8 8 8 8 8 2 6]
 [2 6 6 6 7 6 4 5 8 2 2 3 3 1 1 2 5 3 6 9]
 [8 8 8 2 8 8 2 8 2 9 8 8 8 8 8 2 8 8 8 8]
 [0 6 6 6 6 6 7 5 2 1 7 4 5 6 6 6 7 6 6 4]
 [7 0 6 5 4 6 9 3 3 3 2 1 6 6 5 5 5 6 6 6]
 [0 8 8 3 8 8 2 9 8 2 8 8 8 8 8 2 6 7 6 6]
 [8 8 8 8 9 8 2 6 7 0 8 2 6 7 0 8 8 2 8 2]
 [6 1 2 1 0 6 5 6 6 5 5 8 3 3 2 6 7 4 5 6]
 [0 2 4 1 1 6 7 5 4 5 9 2 2 3 3 1 2 7 7 3]
 [5 6 6 6 6 4 5 9 4 1 1 6 5 4 5 8 3 3 3 1]
 [0 3 3 2 2 7 4 5 5 6 6 4 4 9 0 5 6 5 5 4]
 [6 6 5 4 2 1 5 6 5 4 2 3 2 2 0 7 7 7 6 3]
 [1 6 6 6 5 7 5 5 4 6 6 6 4 5 9 1 7 7 6 4]
 [4 5 1 9 0 8 8 2 7 2 8 8 8 2 8 8 8 9 2 8]
 [1 0 6 6 7 7 6 7 6 5 5 4 6 0 3 2 3 1 1 6]
 [7 0 8 2 8 2 9 8 2 8 2 8 2 8 8 2 8 2 6 7]
 [3 2 8 2 6 6 7 9 8 1 8 2 6 7 0 8 8 2 6 7]
 [5 6 6 6 7 6 7 6 5 5 9 3 2 1 6 5 6 6 6 5]
 [2 8 8 8 8 2 9 2 8 2 8 8 8 8 8 2 6 6 7 0]
 [0 3 1 4 1 2 1 6 4 6 6 7 4 5 9 0 1 5 7 6]]

True classes
 [0 1 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1]
[[[0.01111111 0.18888889 0.18888889 0.18888889 0.18888889 0.0

In [14]:
p,a,b = train(p_c, A, B, start_prob, -np.Inf)
form_clusters(get_targets(cal_resp(p, a, b, start_prob)))

-1717.3256601508238
-1623.8926394457576
-1549.6422005758186
-1545.8690872338848
-1543.2750572880723
-1539.1203541080658
-1532.5777202964932
-1522.3905215829736
-1506.826416590932
-1483.622422646829
-1449.4970359035
-1400.2204317456717
-1336.0927197895403
-1269.7456141251505
-1211.5940119469228
-1154.060994212141
-1089.0732082652698
-1020.4277466751755
-954.8419435834351
-897.4998454927066
-852.7864090028701
-818.5259798954253
-790.3955192739662
-765.9185990867079


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


-743.198385796246


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: RuntimeWarning: divide by zero encountered in log


-720.2316807168464
-699.5006171493553
-682.7292688087105
-668.7325465086105
-656.7640807527721
-646.4863523544122
-637.7983879173373
-630.7452937752491
-624.9147660743128
-619.8761920464974
-615.4665914964589
-611.6421156587618
-608.3759649909996
-605.5103507074697
-602.6072956247115
-599.496062811503
-596.5517794212269
-593.9536570834575
-591.7297619132419
-589.8346535057337
-588.173688633737
-586.6655861918863
-585.254789226484
-583.9050115468366
-582.593956518638
-581.309539739251
-580.0467942657469
-578.8054567421351
-577.5882201644876
-576.3994715687345
-575.2442505912029
-574.1271518008657
-573.0509581901331
-572.0150367980382
-571.0140711452837
-570.038341760398
-569.0764794411568
-568.1193008171194
-567.1605931421061
-566.1916986934227
-565.1908977150417
-564.1092964461186
-562.8510536486966
-561.249004303169
-559.0766814118325
-556.2240818697419
-553.0134775517686
-550.0507796916758
-547.6275954494399
-545.6487479176592
-543.8775687536788
-542.0472831073
-539.8781808012224
-53

{0: [array([ 0,  2,  5,  6, 13, 15, 16, 18], dtype=int64)],
 1: [array([ 1,  3,  4,  7,  8,  9, 10, 11, 12, 14, 17, 19], dtype=int64)]}

In [15]:
form_clusters(targets)

{0: [array([ 0,  2,  5,  6, 13, 15, 16, 18], dtype=int64)],
 1: [array([ 1,  3,  4,  7,  8,  9, 10, 11, 12, 14, 17, 19], dtype=int64)]}

In [8]:
class_prob

array([0.5, 0.5])